In [1]:
%reload_ext autoreload
%autoreload 2

import asyncio
import json
import nest_asyncio
import os
import sys
from dotenv import load_dotenv
import numpy as np
import pandas as pd

sys.path.append('../')
from lattereview.providers.openai_provider import OpenAIProvider
from lattereview.providers.ollama_provider import OllamaProvider
from lattereview.providers.litellm_provider import LiteLLMProvider
from lattereview.agents.scoring_reviewer import ScoringReviewer
from lattereview.review_workflow import ReviewWorkflow

/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Failed to update token costs. Using static costs.
/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/tokencost/constants.py:69: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  logger.error("Failed to update token costs. Using static costs.")


## Setting up the notebook

Loading environment variables:

In [2]:
# Load environment variables from .env file
load_dotenv('../.env')

# Enable asyncio in Jupyter
nest_asyncio.apply()

Loading a dummy dataset:

In [3]:
data = pd.read_csv('test_article_data.csv')
data

,Title,Abstract,Authors,Year
0,Fusing an agent-based model of mosquito popula...,The mosquito Aedes aegypti is the vector of a ...,"Cavany, S.M.",2022
1,PDRL: Multi-Agent based Reinforcement Learning...,Reinforcement learning has been increasingly a...,"Shaik, T.",2023
2,Learning-accelerated Discovery of Immune-Tumou...,We present an integrated framework for enablin...,"Ozik, J.",2019
3,Investigating spatiotemporal dynamics and sync...,"In this paper we present AceMod, an agent-base...","Cliff, O.M.",2018
4,Modeling the Spread of COVID-19 in University ...,Mathematical and simulation models are often u...,"Herrmann, J.W.",2024
5,Multi-Agent Reinforcement Learning with Action...,Unmanned Aerial Vehicles (UAVs) are increasing...,"Rizvi, D.",2023
6,A new (N) stochastic quenched disorder model f...,Human beings live in a networked world in whic...,"Ferreira, A.A.",2019
7,Frustration induced phases in migrating cell c...,Collective motion of cells is common in many p...,"Copenhagen, K.",2017
8,Universal masking is urgent in the COVID-19 pa...,We present two models for the COVID-19 pandemi...,"Kai, D.",2020
9,Calculus of consent via MARL: Legitimating the...,"Public policies that supply public goods, espe...","Hu, Y.",2021


## Testing the base functionalities

Testing the OpenAI provider (with OpenAI and Gemini models):

In [4]:
openanai_provider = OpenAIProvider(model="gpt-4o-mini")
# openanai_provider = OpenAIProvider(model="gemini-1.5-flash")
question = "What is the capital of France?"
asyncio.run(openanai_provider.get_response(question))

('The capital of France is Paris.',
 {'input_cost': 1.05e-06, 'output_cost': 4.2e-06, 'total_cost': 5.25e-06})

Testing the Ollama provider:

In [9]:
ollama_provider = OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434")
question = "What is the capital of France?"
asyncio.run(ollama_provider.get_response(question))

('The capital of France is Paris!',
 {'input_cost': 0, 'output_cost': 0, 'total_cost': 0})

Testing the LiteLLM provider:

In [10]:
# litellm_provider = LiteLLMProvider(model="gpt-4o-mini")
# litellm_provider = LiteLLMProvider(model="claude-3-5-sonnet-20240620")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")
# litellm_provider = LiteLLMProvider(model="ollama/llama3.2-vision:latest")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")
litellm_provider = LiteLLMProvider(model="gemini/gemini-1.5-flash")

question = "What is the capital of France?"
asyncio.run(litellm_provider.get_response(question))

('The capital of France is Paris.\n', 3.45e-06)

Testing the ScoringReviewer agent:

In [11]:
agent = ScoringReviewer(
    # provider=OpenAIProvider(model="gpt-4o-mini"),
    # provider=OpenAIProvider(model="gemini-1.5-flash"),
    # provider=OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434"),
    # provider=LiteLLMProvider(model="gpt-4o-mini"),
    provider=LiteLLMProvider(model="claude-3-5-sonnet-20241022"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    # provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Pouria",
    max_concurrent_requests=1, 
    backstory="an expert reviewer and researcher!",
    input_description = "article title",
    model_args={"max_tokens": 100, "temperature": 0.1},
    reasoning = "brief",
    review_criteria="Look for articles that certainly do not employ any AI or machine learning agents",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper does not meet the criteria, and 2 if the paper meets the criteria.',
)


# Dummy input
text_list = data.Title.str.lower().tolist()
print("Inputs:\n\n", '\n'.join(text_list[:3]), "\n\n")

# Dummy review
results, total_cost = asyncio.run(agent.review_items(text_list[:3]))
print("Outputs:")
for result in results:
    print(result)

# Dummy costs
print("\nCosts:\n")
for item in agent.memory:
    print(item['cost'])

print("\nTotal cost:\n")
print(total_cost)

Inputs:

 segmentized quarantine policy for managing a tradeoff between containment of infectious disease and social cost of quarantine
autoproteinengine: a large language model driven agent framework for multimodal automl in protein engineering
integration of large vision language models for efficient post-disaster damage assessment and reporting 




Reviewing 3 items - 2024-12-14 22:24:13: 100%|██████████| 3/3 [00:07<00:00,  2.43s/it]

Outputs:
{'reasoning': 'The title clearly indicates a policy-focused study addressing the balance between disease containment and social impacts through quarantine segmentation, which is a relevant and well-defined research topic.', 'score': 2}
{'reasoning': 'The title suggests a novel framework combining LLMs with AutoML for protein engineering, indicating a clear and relevant contribution to automated machine learning applications.', 'score': 2}
{'reasoning': 'The title effectively indicates the use of vision-language models for post-disaster assessment, which is a clear and relevant application of AI for disaster management.', 'score': 2}

Costs:

0.003042
0.002931
0.002925

Total cost:

0.002925


## Testing the main Functionalities

#### A multiagent review workflow for doing title/abstract analysis

Setting up the agents:

In [12]:
pouria = ScoringReviewer(
    # provider=OpenAIProvider(model="gemini-1.5-flash"),
    # provider=OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Pouria",
    max_concurrent_requests=20, 
    backstory="a radiologist with many years of background in statistcis and data science, who are famous among your colleagues for your systematic thinking, organizaton of thoughts, and being conservative",
    model_args={"max_tokens": 100, "temperature": 0.1},
    input_description = "tilte and abstract of scientific articles",
    reasoning = "cot",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

bardia = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o-mini"),
    name="Bardia",
    max_concurrent_requests=20, 
    backstory="an expert in data science with a background in developing ML models for healthcare, who are famous among your colleagues for your creativity and out of the box thinking",
    model_args={"max_tokens": 100, "temperature": 0.8},
    input_description = "tilte and abstract of scientific articles",
    reasoning = "brief",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

brad = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o"),
    name="Brad",
    max_concurrent_requests=20, 
    backstory="a senior radiologist with a PhD in computer science and years of experience as the director of a DL lab focused on developing ML models for radiology and healthcare",
    input_description = "tilte and abstract of scientific articles",
    temperature=0.4,
    reasoning = "cot",
    max_tokens=100,
    scoring_task="""Pouria and Bardia have Looked for articles that disucss large languange models-based AI agents applied to medical imaging data. 
                       They scored an article 1 if they thought it does not meet this criteria, 2 if they thought it meets the criteria, 0 if they were uncertain of scoring.
                       You will receive an article they have had different opinions about, as well as each of their scores and their reasoning for that score. Read their reviews and determine who you agree with. 
                    """,
    score_set=[1, 2],
    scoring_rules="""Score 1 if you agree with Pouria, and score 2 if you agree with Bardia.""",
)


Setting up the review workflow:

In [13]:
title_abs_review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [pouria, bardia],
            "inputs": ["Title", "abstract"]
        },
        {
            "round": 'B',
            "reviewers": [brad],
            "inputs": ["Title", "abstract", "round-A_Pouria_output", "round-A_Bardia_output"],
            "filter": lambda row: row["round-A_Pouria_output"]["score"] != row["round-A_Bardia_output"]["score"]
        }
    ]
)

Applying the review workflow to a number of sample articles:

In [14]:
# Reload the data if needed.
sample_data = pd.read_excel('data.xlsx').sample(10).reset_index(drop=True)
updated_data = asyncio.run(title_abs_review(sample_data))

print("Total cost: ")
print(title_abs_review.get_total_cost())

print("\nDetailed cost:")
print(title_abs_review.reviewer_costs)

updated_data


Starting review round A (1/2)...
Processing 10 eligible rows


['round: A', 'reviewer_name: Pouria'] -                     2024-12-14 22:24:20: 100%|██████████| 10/10 [00:01<00:00,  8.03it/s]
['round: A', 'reviewer_name: Bardia'] -                     2024-12-14 22:24:22: 100%|██████████| 10/10 [00:01<00:00,  6.55it/s]


Starting review round B (2/2)...
Skipping review round B - no eligible rows
Total cost: 
0.000132

Detailed cost:
{('A', 'Pouria'): 5.01e-05, ('A', 'Bardia'): 8.19e-05}


,ID,Title,1st author,repo,year,abstract,round-A_Pouria_output,round-A_Pouria_score,round-A_Pouria_reasoning,round-A_Bardia_output,round-A_Bardia_score,round-A_Bardia_reasoning
0,19,JAILJUDGE: A COMPREHENSIVE JAILBREAK JUDGE BEN...,"Liu, F.",arXiv,2024,Although significant research efforts have bee...,{'reasoning': 'The abstract does not mention m...,2,The abstract does not mention medical imaging ...,{'reasoning': 'The article focuses on evaluati...,2,The article focuses on evaluating the safety a...
1,85,An Autonomous GIS Agent Framework for Geospati...,"Ning, H.",arXiv,2024,Powered by the emerging large language models ...,{'reasoning': 'The article discusses an autono...,2,The article discusses an autonomous GIS agent ...,{'reasoning': 'The paper discusses an autonomo...,2,The paper discusses an autonomous GIS agent fr...
2,434,Bifurcation theory captures band formation in ...,"Trenado, C.",arXiv,2022,Collective behavior occurs ubiquitously in nat...,{'reasoning': 'The provided abstract does not ...,2,The provided abstract does not discuss large l...,{'reasoning': 'This article does not discuss l...,2,This article does not discuss large language m...
3,191,A Call for Embodied AI,"Paolo, G.",arXiv,2024,We propose Embodied AI (E-AI) as the next fund...,{'reasoning': 'The abstract mentions Large Lan...,2,The abstract mentions Large Language Models (L...,{'reasoning': 'The article focuses on Embodied...,2,The article focuses on Embodied AI and its the...
4,524,Learning from zero: how to make consumption-sa...,"Shi, R.",arXiv,2021,This exercise proposes a learning mechanism to...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article focuses on economic...,2,The article focuses on economic decision-makin...
5,251,Individual Variation Affects Outbreak Magnitud...,"Lazebnik, T.",arXiv,2023,Zoonotic disease transmission between animals ...,{'reasoning': 'The article focuses on a multi-...,2,The article focuses on a multi-pathogen model ...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...
6,707,Agent-based modeling predicts HDL-independent ...,"Paalvast, Y.",bioRxiv,2018,A hallmark of the metabolic syndrome is low HD...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article discusses an agent-...,2,The article discusses an agent-based modeling ...
7,231,Moderate confirmation bias enhances collective...,"Bergerot, C.",bioRxiv,2023,Humans tend to give more weight to information...,{'reasoning': 'The provided abstract does not ...,2,The provided abstract does not discuss large l...,{'reasoning': 'The article discusses reinforce...,2,The article discusses reinforcement-learning a...
8,755,SPEW: Synthetic populations and ecosystems of ...,"Gallagher, S.",arXiv,2017,Agent-based models (ABMs) simulate interaction...,{'reasoning': 'The abstract does not mention l...,2,The abstract does not mention large language m...,{'reasoning': 'The article discusses agent-bas...,2,The article discusses agent-based models for s...
9,362,Neural Network Augmented Compartmental Pandemi...,"Kummer, L.",arXiv,2022,Compartmental models are a tool commonly used ...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article focuses on neural n...,2,The article focuses on neural network augmente...


In [15]:
for i, row in updated_data.iterrows():
    print(
        f"""
        Title: {row.Title}
        Abstract: {row.abstract}
        Pouria's score: {row["round-A_Pouria_score"]}
        Pouria's reasoning: {row["round-A_Pouria_reasoning"]}
        Bardia's score: {row["round-A_Bardia_score"]}
        Bardia's reasoning: {row["round-A_Bardia_reasoning"]}
        Brad's score: {None if "round-B_Brad_score" not in row else row["round-B_Brad_score"]}
        Brad's reasoning: {None if "round-B_Brad_reasoning" not in row else row["round-B_Brad_reasoning"]}
        """
    )


        Title: JAILJUDGE: A COMPREHENSIVE JAILBREAK JUDGE BENCHMARK WITH MULTI-AGENT ENHANCED EXPLANATION EVALUATION FRAMEWORK
        Abstract: Although significant research efforts have been dedicated to enhancing the safety of large language models (LLMs) by understanding and defending against jailbreak attacks, evaluating the defense capabilities of LLMs against jailbreak attacks also attracts lots of attention. Current evaluation methods lack explainability and do not generalize well to complex scenarios, resulting in incomplete and inaccurate assessments (e.g., direct judgment without reasoning explainability, the F1 score of the GPT-4 judge is only 55% in complex scenarios and bias evaluation on multilingual scenarios, etc.). To address these challenges, we have developed a comprehensive evaluation benchmark, JAILJUDGE, which includes a wide range of risk scenarios with complex malicious prompts (e.g., synthetic, adversarial, in-the-wild, and multi-language scenarios, etc.) alo